In [ ]:
import sys  
sys.path.insert(0, '/Users/johanneswidera/Uni/bachelorarbeit/Code/models/')

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import shap
import pandas as pd
import numpy as np
from sklearn import linear_model
from bs4 import BeautifulSoup
import os
import contractions
import re
import string
from helper import read_imdb_split,download_data
from model_helper.tf_idf import build_tf_idf
from model_helper.PipelineWrapper import PipelineWrapper
from custom_shap_explainer.custom_global import custom_global_explanation
from custom_shap_explainer.signal_words import highlight_signal_words
np.random.seed(1337)
shap.initjs()

# 1. Get Training Data 

In [ ]:
download_data()
corpus_train, y_train = read_imdb_split('../data/aclImdb/train')
corpus_test, y_test = read_imdb_split('../data/aclImdb/test')


# 2. Build Vectorizer

In [ ]:
vectorizer_tf_idf = build_tf_idf()
vectorizer_tf_idf.fit(corpus_train)

# 3. Build Models

# 3.1 Build Logistic Regression Model

In [ ]:
model = linear_model.LogisticRegression(penalty="l2")
model_logregression_new = PipelineWrapper(model, vectorizer_tf_idf, corpus_test, corpus_train, y_test, y_train)

In [ ]:
model_logregression_new.fit()

In [ ]:
model_logregression_new.report()

In [ ]:
predictions_logreg = model_logregression_new.predict_proba(corpus_test)


LOGREG_PREDICTION_FILE = 'predictions_logreg.csv'

file_exists = os.path.exists(LOGREG_PREDICTION_FILE)

if not file_exists:
    # Calculate BERT predictions
    
    predictions_logreg = model_logregression_new.predict_proba(corpus_test)

# Destructure probabilities for class_1 and class_2
    predictions_class_0 = predictions_logreg[:, 0]
    predictions_class_1 = predictions_logreg[:, 1]
    
    # Create a DataFrame containing the test samples and BERT predictions
    results = pd.DataFrame({
    'logreg_0': predictions_class_0,
    'logreg_1': predictions_class_1
    })
    # Create a DataFrame containing the test samples and BERT predictions
  
    # Save the DataFrame to a CSV file
    results.to_csv(LOGREG_PREDICTION_FILE)

# 3.2 Build Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier


model_dtc = PipelineWrapper(DecisionTreeClassifier(min_samples_leaf=20), vectorizer_tf_idf, corpus_test, corpus_train, y_test, y_train)

In [ ]:
model_dtc.fit()

In [ ]:
model_dtc.report()

In [ ]:
predictions_DTC = model_dtc.predict_proba(corpus_test[:1])
predictions_DTC

In [ ]:
DTC_PREDICTION_FILE = 'predictions_dtc.csv'

file_exists = os.path.exists(DTC_PREDICTION_FILE)

if not file_exists:
    # Calculate DTC predictions
    predictions_DTC = model_dtc.predict_proba(corpus_test)

# Destructure probabilities for class_1 and class_2
    predictions_class_0 = predictions_DTC[:, 0]
    predictions_class_1 = predictions_DTC[:, 1]
    
    # Create a DataFrame containing the test samples and BERT predictions
    results = pd.DataFrame({
    'dtc_0': predictions_class_0,
    'dtc_1': predictions_class_1
    })
    results.to_csv(DTC_PREDICTION_FILE)

# 3.3 Build Fine Tuned BERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("../../FineTunedBERT/Sentiment/24060142")
# Load model
loaded_model = AutoModelForSequenceClassification.from_pretrained("../../FineTunedBERT/Sentiment/24060142")
model_bert = pipeline('sentiment-analysis', model=loaded_model, tokenizer=tokenizer, max_length=512, truncation=True, top_k=None) 

In [ ]:
model_bert.predict(corpus_test[:5])

In [ ]:
BERT_PREDICTION_FILE = 'prediction_BERT.csv'

file_exists = os.path.exists(BERT_PREDICTION_FILE)

if not file_exists:
    # Calculate BERT predictions
    raw_predictions_bert = model_bert.predict(corpus_test)

    # Destructure probabilities for class_1 and class_2
    predictions_class_0 = [pred[0]['score'] if pred[0]['label'] == 'LABEL_0' else pred[1]['score'] for pred in raw_predictions_bert]
    predictions_class_1 = [pred[0]['score'] if pred[0]['label'] == 'LABEL_1' else pred[1]['score'] for pred in raw_predictions_bert]

    # Create a DataFrame containing the test samples and BERT predictions
    results = pd.DataFrame({
        'bert_0': predictions_class_0,
        'bert_1': predictions_class_1
    })

    # Save the DataFrame to a CSV file
    results.to_csv(BERT_PREDICTION_FILE)


# 3.4 load all prediciton in one dataframe
so we can see which sample is most interesting 

In [ ]:
from Sentiment.disagreements import get_disagreements

dtc_bert_sorted_by_diff,logreg_bert_sorted_by_diff,logreg_dtc_sorted_by_diff = get_disagreements(y_test,BERT_PREDICTION_FILE, LOGREG_PREDICTION_FILE, DTC_PREDICTION_FILE)

# Wähle die Top-N interessanten Samples
top_n = 3

In [ ]:
dtc_bert_sorted_by_diff.head(top_n)

In [ ]:
logreg_bert_sorted_by_diff.head(top_n)

In [ ]:
logreg_dtc_sorted_by_diff.head(top_n)

# 4 Analyze With SHAP

We know that the Logistic Regression Performs better than the Decision Tree Classifier.
To further investigate why thats the case we need to look into the models.

In the following i will do 3 things.

Per Model:
1. Most Positive/Negative Words
2. Investigate the Most Wrong Positive and Negative prediction

Model Comparision:
1. Investigate biggest Prediction gap accross the models

# 4.1 Logistic Regression with SHAP

In [ ]:
masker = shap.maskers.Text(tokenizer=r"\W+") # this will create a basic whitespace tokenizer
# explainer_logreg = shap.Explainer(model_logregression_new.predict_proba , masker)
explainer_logreg = shap.Explainer(model_logregression_new.predict_proba, masker)
#shap_values_logreg = explainer_logreg(corpus_test)


In [ ]:
# Performance Optimazation

import os.path
import pickle

shap_values_file = "shap_values_logreg.pkl"

# Überprüfen, ob die Datei existiert
if not os.path.exists(shap_values_file):
    # Berechnen Sie die SHAP-Werte, wenn die Datei nicht existiert
    shap_values_logreg = explainer_logreg(corpus_test)

    # Speichern Sie die SHAP-Werte in einer Datei
    with open(shap_values_file, "wb") as f:
        pickle.dump(shap_values_logreg, f)
else:
    # Laden Sie die SHAP-Werte aus der Datei, wenn sie existiert
    with open(shap_values_file, "rb") as f:
        shap_values_logreg = pickle.load(f)



### 4.1 Investigate the biggest misclassification


In [ ]:
from Sentiment.disagreements import get_misclassifications
logreg_misclassifications = get_misclassifications(y_test,LOGREG_PREDICTION_FILE)

In [ ]:
logreg_misclassifications[:5]

In [ ]:
# Really good example to investigate why the model predicted the wrong class
most_wrong_positive_index  = 9634

#### 4.1.2 Most Wrong Positive Classification

Most wrong review 

In [ ]:
print(logreg_misclassifications.loc[most_wrong_positive_index])
print(corpus_test[most_wrong_positive_index])

In [ ]:
explanation = shap_values_logreg[most_wrong_positive_index, :, 1]

so its 0.5 behind the real label.
But Why lets investigate

In [ ]:
shap.plots.text(explanation)
shap.plots.waterfall(explanation)

In [ ]:
# further investigate what words really matter - highlight the words that contributed most to the prediction

highlight_signal_words(explanation,round_shap_values=3,top_words=4)

# interpretation:

The model learned that the word bad and boring are negative sentiment words and they overlap all other words you can see that the review is really positive beacuse of this passage:
`after the slow beginning and some of the soap opera antics i started liking it the plot was different than anything i had ever seen now`.

So it wasnt able to understand that the word liking is really important for the sentiment.

With the highlight Plot we can really see that it doesnt pay relevant attention to the relevant part.

#### 4.1.2 Most Wrong Negative Classification

Most wrong review 

In [ ]:
most_wrong_negative_index  = 8426
explanation = shap_values_logreg[most_wrong_negative_index, :, 1]

In [ ]:
print(logreg_misclassifications.loc[most_wrong_negative_index])
print(corpus_test[most_wrong_negative_index])

In [ ]:
shap.plots.text(explanation)
shap.plots.waterfall(explanation)

In [ ]:
highlight_signal_words(explanation,round_shap_values=3,top_words=4)

# Interpretation:

This was a correct classification but the initial label is wrong?
So we found a mistake in the labeled training data.
With this information we can remove the sample and further improve the training process


# 4.2 Global Interpretation Most Positive and negative Words

To understand how the model works

In [ ]:
custom_global_explanation(shap_values_logreg[:,:,1])

# interpretation:

It seems like the model learned the correct words

# 5. Analyze with SHAP: DTC

In [ ]:
masker = shap.maskers.Text(tokenizer=r"\W+") # this will create a basic whitespace tokenizer
# explainer_logreg = shap.Explainer(model_logregression_new.predict_proba , masker)
explainer_dtc = shap.Explainer(model_dtc.predict_proba, masker)

In [ ]:
import os.path
import pickle

shap_values_file = "shap_values_dtc.pkl"

# Überprüfen, ob die Datei existiert
if not os.path.exists(shap_values_file):
    # Berechnen Sie die SHAP-Werte, wenn die Datei nicht existiert
    shap_values_dtc= explainer_dtc(corpus_test)

    # Speichern Sie die SHAP-Werte in einer Datei
    with open(shap_values_file, "wb") as f:
        pickle.dump(shap_values_dtc, f)
else:
    # Laden Sie die SHAP-Werte aus der Datei, wenn sie existiert
    with open(shap_values_file, "rb") as f:
        shap_values_dtc = pickle.load(f)

# 5.1 Investigate Biggest Misclasssifications

In [ ]:
dtc_misclassifications = get_misclassifications(y_test, DTC_PREDICTION_FILE)

In [ ]:
dtc_misclassifications[:5]


In [ ]:
most_wrong_positive_index = 11617

### 5.2.1 Most Wrong Positive Classification

In [ ]:
explanation = shap_values_dtc[most_wrong_positive_index, :, 1]
print(dtc_misclassifications.loc[most_wrong_positive_index])
print(corpus_test[most_wrong_positive_index])

In [ ]:
model_dtc.predict_proba([corpus_test[most_wrong_positive_index]])

In [ ]:
print(explanation.base_values)
shap.plots.text(explanation)
shap.plots.waterfall(explanation)

In [ ]:
highlight_signal_words(explanation,round_shap_values=3,top_words=4)

### interpretation

if you read it you know This is a positive Review. 
This Classifier Fails to understand that the review is talking about the scene and not about the movie
you can clearly see the dominating words food they have look are all understand as negative words have great impact on the result with Shap values up to -0.2

### 5.2.2 Most Wrong Negative Classification

In [ ]:
most_wrong_negative_index = 11488
explanation = shap_values_dtc[most_wrong_negative_index, :, 1]
print(dtc_misclassifications.loc[most_wrong_negative_index])
print(corpus_test[most_wrong_negative_index])



In [ ]:
shap.plots.text(explanation)
shap.plots.waterfall(explanation)
highlight_signal_words(explanation,round_shap_values=3,top_words=4)

#### Interpretation 

This is clearly a negative review this person hates the movie.
What confused the model was the use of great it was used in respect of the talent of the actor and what could have saved the movie: "great momments of comedy dark humour".
But both uses of great wasnt about the movie.

# 5.3 Global Explanation

In [ ]:

custom_global_explanation(shap_values_dtc[:,:,1])


#### interpretation:

you can directly see the Shap values Words with a Positive impact in the 1 percentile are very low and many words. you can directly see that the negative words only 4 lead the sentiment so mess even worse and crap this is great

In [ ]:
custom_global_explanation(shap_values_dtc[:,:,1])

# 6 SHAP for BERT

In [ ]:
""" 

In summary, the reason your code is not using the GPU is that the SHAP library does not support GPU acceleration for BERT models. Unfortunately, there is no direct solution to this issue. You can try looking for alternative libraries or methods that support GPU acceleration for BERT or similar text models.
 """

# Performance Optimazation
masker = shap.maskers.Text(tokenizer=r"\W+") # this will create a basic whitespace tokenizer
# explainer_logreg = shap.Explainer(model_logregression_new.predict_proba , masker)
explainer_bert = shap.Explainer(model_bert, masker)
import os.path
import pickle

shap_values_file = "shap_values_bert.pkl"

# Überprüfen, ob die Datei existiert
if not os.path.exists(shap_values_file):
    # Berechnen Sie die SHAP-Werte, wenn die Datei nicht existiert
    shap_values_bert= explainer_bert(corpus_test[:100])

    # Speichern Sie die SHAP-Werte in einer Datei
    with open(shap_values_file, "wb") as f:
        pickle.dump(shap_values_bert, f)
else:
    # Laden Sie die SHAP-Werte aus der Datei, wenn sie existiert
    with open(shap_values_file, "rb") as f:
        shap_values_bert = pickle.load(f)

In [ ]:
shap_values_bert.shape

# 6.1 Investigate Biggest Misclassifications



In [ ]:
dtc_misclassifications = get_misclassifications(y_test, BERT_PREDICTION_FILE)
dtc_misclassifications[:10]

### 6.1.2 Biggest Positive Misclassification

In [ ]:
most_wrong_positive_index = 19396

explanation = explainer_bert([corpus_test[most_wrong_positive_index]])
explanation = explanation[0, :, 1]

In [ ]:
print(dtc_misclassifications.loc[most_wrong_positive_index])
print(corpus_test[most_wrong_positive_index])
shap.plots.text(explanation)
shap.plots.waterfall(explanation)

In [ ]:
highlight_signal_words(explanation,round_shap_values=3,top_words=4)

#### Interpretration

This Classification was correct the overall sentiment of the review is negative.
But it sounds like this was a disappointed fan who is disappointed how the last laurel and hardy movie ended.
Nevertheless its resulted in a review with a score of > 7.
This could be fixed in the original dataset.



### 6.1.2 Biggest Negative Misclassification 

In [ ]:
most_wrong_negative_index = 3556

explanation = explainer_bert([corpus_test[most_wrong_negative_index]])
explanation = explanation[0, :, 1]

In [ ]:
print(dtc_misclassifications.loc[most_wrong_negative_index])
print(corpus_test[most_wrong_negative_index])
shap.plots.text(explanation)
shap.plots.waterfall(explanation)

In [ ]:
highlight_signal_words(explanation,round_shap_values=3,top_words=4)

#### Interpretation

Man sieht klar das die wörter mit Positiven Sentiment einen Grossen einfuss auf das Review haben:
enjoyed, impressed, incredible, great.
und das das Review auch eindeutig Positiv ist

Jedoch ist es trotzdem Fehlclassifikation nach weiterer Rechereche findet man das original review unter <a href="https://www.imdb.com/review/rw2198364/?ref_=tt_urv">REVIEW</a>.
Und es wurde eindeutig die Falsche anzahl an sternen vergeben

# 6.2 Global Explanation

In [ ]:
custom_global_explanation(shap_values_bert[:,:,1])

# 7 Model Comparision

## DTC vs BERT

In [ ]:
dtc_bert_sorted_by_diff.head(5)

## logreg vs BERT

In [ ]:
logreg_bert_sorted_by_diff.head(5)

In [ ]:
bert_dtc_log_reg_biggest_difference_index = 7153



In [ ]:
corpus_test[bert_dtc_log_reg_biggest_difference_index]

In [ ]:
dtc_explanation = explainer_dtc([corpus_test[bert_dtc_log_reg_biggest_difference_index]])
logreg_explanation = explainer_logreg([corpus_test[bert_dtc_log_reg_biggest_difference_index]])

In [ ]:
print('DTC Explanation')
highlight_signal_words(dtc_explanation[0,:,1],round_shap_values=3,top_words=4)
shap.plots.text(dtc_explanation[0,:,1])

In [ ]:
print('logreg Explanation')
highlight_signal_words(logreg_explanation[0,:,1],round_shap_values=3,top_words=4)
shap.plots.text(logreg_explanation[0,:,1])

In [ ]:
bert_explanation = explainer_bert([corpus_test[bert_dtc_log_reg_biggest_difference_index]])


In [ ]:
bert_explanation = bert_explanation[0, :, 1]

In [ ]:
print('BERT Explanation')
highlight_signal_words(bert_explanation,round_shap_values=3,top_words=4)
shap.plots.text(bert_explanation)
shap.plots.waterfall( bert_explanation)

## interpretation:

The Worst Classification was Logistic Regression lets investigate:
Again an example were there reviewer is talking about the movie content and this is interpreted as Negative sentiment.
The model is not contributing anough focus on the last line overal its a good movie.

The second Worst Classification was Decision Tree Classifier lets investigate:
here its more like a 50/50 Decision the model cant really decide if its negative or positive
the most important positive word was "best" and most important negative "Directing was bad"

The best Classification was BERT:
the biggest focus was on 7 so it understand as soon a 7 is present its like 7/10 so its a positive review.
altough it was cheap (score: -0.26) shap understand the the reviewer thinks its overall a good movie


